# Import Data

In [ ]:
# import libraries
import pandas as pd                                   # data science essentials
import matplotlib.pyplot as plt                       # data visualization
import seaborn as sns                                 # enhanced data visualization
import numpy as np                                    # contruct arrays
import statsmodels.formula.api as smf                 # statsmodel: OLS model 
from sklearn.model_selection import train_test_split  # scikit-learn: train/test split 
from sklearn.linear_model import LinearRegression     # scikit-learn: linear models 

# set pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# specify the path and file name
file = './datasets/Apprentice_Chef_Dataset.xlsx'

# read the file into Python
df = pd.read_excel(io=file)

# EXPLORE DATA

In [ ]:
# 1. View number of rows and columns in dataset 
df.shape

#format and print output (0 = rows, 1 = columns)
print(f"""
Size of Original Dataset
Observations: {df.shape[0]}      
Features:     {df.shape[1]}      
""")

In [ ]:
# 2. Check info of each variable (focusing on data type for grouping)
df.info()

# ENGINEER FEATURES 

In [ ]:
# HOW EACH VARIABLE WAS ENGINEERED: 

#OBJECTS
NAME                         
EMAIL                     #split into groups (personal, professional, junk)   
FIRST_NAME                 
FAMILY_NAME               

#CONTINUOUS 
REVENUE                   #y-variable (Log)
TOTAL_MEALS_ORDERED       #Log                 
AVG_TIME_PER_SITE_VISIT   #Log 
AVG_PREP_VID_TIME         #Log 
TOTAL_PHOTOS_VIEWED       #flag 

#INTERVAL/COUNT 
CROSS_SELL_SUCCESS          #0-1 (no engineer)
MOBILE_NUMBER               #0-1 (no engineer)
TASTES_AND_PREFERENCES      #0-1 (no engineer) 
WEEKLY_PLAN                 #flag 
UNIQUE_MEALS_PURCH          #Log     
CONTACTS_W_CUSTOMER_SERVICE #Log
LARGEST_ORDER_SIZE          #Log
AVG_CLICKS_PER_VISIT        #Log
PC_LOGINS                   #Log 
PRODUCT_CATEGORIES_VIEWED   #Log 
MOBILE_LOGINS               #(no engineer) 
CANCELLATIONS_BEFORE_NOON   #flag --> Total Cancellations
CANCELLATIONS_AFTER_NOON    #flag --> Total Cancellations
EARLY_DELIVERIES            #flag --> Total Deliveries
LATE_DELIVERIES             #flag --> Total Deliveries 
REFRIGERATED_LOCKER         #Total Locker
PACKAGE_LOCKER              #Total Locker

#CATEGORICAL
MEDIAN_MEAL_RATING #Label 1-2-3-4-5 and get dummy 
MASTER_CLASSES_ATTENDED #Label 0-1-2-3 and get dummy 

## Explore and Transform Y-Variable

In [ ]:
# Step 1: develop a histogram for REVENUE
sns.displot(data   = df,
                x  = df.loc[:,'REVENUE'],
                height = 5,
                aspect = 2)

# display plot 
plt.show()

In [ ]:
# Step 2: transform REVENUE and save it to the dataset
df['log_REVENUE'] = np.log10(df['REVENUE'])

# Step 3: plot to check for normality (no more skewness)
sns.displot(data = df,
            x = 'log_REVENUE',
            height = 5,
            aspect = 2)

# display plot 
plt.show()

## Flag Trend-Based Features

In [ ]:
# Step 1: Count variables with the number 0 
total_meals_zeroes          = len(df['TOTAL_MEALS_ORDERED']
                                  [df['TOTAL_MEALS_ORDERED'] == 0])
cancel_before_noon_zeroes   = len(df['CANCELLATIONS_BEFORE_NOON']
                                  [df['CANCELLATIONS_BEFORE_NOON'] == 0]) 
cancel_after_noon_zeroes    = len(df['CANCELLATIONS_AFTER_NOON']
                                  [df['CANCELLATIONS_AFTER_NOON'] == 0]) 
weekly_plan_zeroes          = len(df['WEEKLY_PLAN']
                                  [df['WEEKLY_PLAN'] == 0]) 
early_deliver_zeroes        = len(df['EARLY_DELIVERIES']
                                  [df['EARLY_DELIVERIES'] == 0]) 
late_deliver_zeroes         = len(df['LATE_DELIVERIES']
                                  [df['LATE_DELIVERIES'] == 0]) 
total_photos_zeroes         = len(df['TOTAL_PHOTOS_VIEWED']
                                  [df['TOTAL_PHOTOS_VIEWED'] == 0]) 

# Step 2: Print a table of the results
print(f"""
                       No\t\tYes
                     ---------------------
Total Meals         | {total_meals_zeroes}\t\t\t{len(df) - total_meals_zeroes}
Cancel Before Noon  | {cancel_before_noon_zeroes}\t\t{len(df) - cancel_before_noon_zeroes}
Cancel After Noon   | {cancel_after_noon_zeroes}\t\t{len(df) - cancel_after_noon_zeroes}
Weekly Plan         | {weekly_plan_zeroes}\t\t{len(df) - weekly_plan_zeroes}
Early Delivery      | {early_deliver_zeroes}\t\t{len(df) - early_deliver_zeroes}
Late Delivery       | {late_deliver_zeroes}\t\t{len(df) - late_deliver_zeroes}
Total Photos Viewed | {total_photos_zeroes}\t\t{len(df) - total_photos_zeroes}


""")

In [ ]:
# Step 3: flag variables with at least 100 obs for 'yes' and 'no' columns
df['HAS_CANCELLATIONS_BEFORE_NOON'] = 0
df['HAS_CANCELLATIONS_AFTER_NOON']  = 0
df['HAS_WEEKLY_PLAN']               = 0
df['HAS_EARLY_DELIVERIES']          = 0
df['HAS_LATE_DELIVERIES']           = 0
df['HAS_TOTAL_PHOTOS_VIEWED']       = 0

# iterate over each original column to change values in the new columns:
for index, value in df.iterrows():   
    
    # CANCELLATIONS_BEFORE_NOON
    if df.loc[index, 'CANCELLATIONS_BEFORE_NOON'] > 0:
        df.loc[index, 'HAS_CANCELLATIONS_BEFORE_NOON'] = 1
        
    # CANCELLATIONS_AFTER_NOON
    if df.loc[index, 'CANCELLATIONS_AFTER_NOON'] > 0:
        df.loc[index, 'HAS_CANCELLATIONS_AFTER_NOON'] = 1
        
    # WEEKLY_PLAN
    if df.loc[index, 'WEEKLY_PLAN'] > 0:
        df.loc[index, 'HAS_WEEKLY_PLAN'] = 1 
        
    # EARLY_DELIVERIES
    if df.loc[index, 'EARLY_DELIVERIES'] > 0:
        df.loc[index, 'HAS_EARLY_DELIVERIES'] = 1   
        
    # LATE_DELIVERIES
    if df.loc[index, 'LATE_DELIVERIES'] > 0:
        df.loc[index, 'HAS_LATE_DELIVERIES'] = 1
        
    # TOTAL_PHOTOS_VIEWED
    if df.loc[index, 'TOTAL_PHOTOS_VIEWED'] > 0:
        df.loc[index, 'HAS_TOTAL_PHOTOS_VIEWED'] = 1   
        
# check results
df[['HAS_CANCELLATIONS_BEFORE_NOON', 'HAS_CANCELLATIONS_AFTER_NOON', 
    'HAS_WEEKLY_PLAN', 'HAS_EARLY_DELIVERIES', 'HAS_LATE_DELIVERIES',
    'HAS_TOTAL_PHOTOS_VIEWED']].head()

In [ ]:
# Step 4: check the correlations of new features with REVENUE

# develop a small correlation matrix
zeroes_corr = df.corr()

# check the correlations of the newly-created variables with REVENUE
zeroes_corr.loc['REVENUE',                                   
               ['HAS_CANCELLATIONS_BEFORE_NOON', 'HAS_CANCELLATIONS_AFTER_NOON',
                'HAS_WEEKLY_PLAN', 'HAS_EARLY_DELIVERIES', 'HAS_LATE_DELIVERIES',
                'HAS_TOTAL_PHOTOS_VIEWED']] \
                .sort_values(ascending = False)

## Transform X-Variables Using Log

In [ ]:
# Step 1: transform original data using log10 
# Step 2: check correlations with Y-variables 
# Step 3: present results to see improvement in transformation 

In [ ]:
# TOTAL_MEALS_ORDERED 

df['log_TOTAL_MEALS_ORDERED'] = np.log10(df['TOTAL_MEALS_ORDERED'])

log_corr = df.loc[ : , ['TOTAL_MEALS_ORDERED',
                        'log_TOTAL_MEALS_ORDERED',
                        'REVENUE',
                        'log_REVENUE']  ].corr(method = 'pearson')\
                                                 .round(decimals = 2)

log_corr.loc[ ['TOTAL_MEALS_ORDERED', 'log_TOTAL_MEALS_ORDERED'],
              ['REVENUE', 'log_REVENUE']   ]

In [ ]:
# AVG_TIME_PER_SITE_VISIT

df['log_AVG_TIME_PER_SITE_VISIT'] = np.log10(df['AVG_TIME_PER_SITE_VISIT'])

log_corr = df.loc[ : , ['AVG_TIME_PER_SITE_VISIT',
                        'log_AVG_TIME_PER_SITE_VISIT',
                        'REVENUE',
                        'log_REVENUE']  ].corr(method = 'pearson')\
                                                 .round(decimals = 2)

log_corr.loc[ ['AVG_TIME_PER_SITE_VISIT', 'log_AVG_TIME_PER_SITE_VISIT'],
              ['REVENUE', 'log_REVENUE']   ]

In [ ]:
# AVG_PREP_VID_TIME

df['log_AVG_PREP_VID_TIME'] = np.log10(df['AVG_PREP_VID_TIME'])

log_corr = df.loc[ : , ['AVG_PREP_VID_TIME',
                        'log_AVG_PREP_VID_TIME',
                        'REVENUE',
                        'log_REVENUE']  ].corr(method = 'pearson')\
                                                 .round(decimals = 2)

log_corr.loc[ ['AVG_PREP_VID_TIME', 'log_AVG_PREP_VID_TIME'],
              ['REVENUE', 'log_REVENUE']   ]

In [ ]:
# UNIQUE_MEALS_PURCH      

df['log_UNIQUE_MEALS_PURCH'] = np.log10(df['UNIQUE_MEALS_PURCH'])

log_corr = df.loc[ : , ['UNIQUE_MEALS_PURCH',
                        'log_UNIQUE_MEALS_PURCH',
                        'REVENUE',
                        'log_REVENUE']  ].corr(method = 'pearson')\
                                                 .round(decimals = 2)

log_corr.loc[ ['UNIQUE_MEALS_PURCH', 'log_UNIQUE_MEALS_PURCH'],
              ['REVENUE', 'log_REVENUE']   ]

In [ ]:
# LARGEST_ORDER_SIZE   

df['log_LARGEST_ORDER_SIZE'] = np.log10(df['LARGEST_ORDER_SIZE'])

log_corr = df.loc[ : , ['LARGEST_ORDER_SIZE',
                        'log_LARGEST_ORDER_SIZE',
                        'REVENUE',
                        'log_REVENUE']  ].corr(method = 'pearson')\
                                                 .round(decimals = 2)

log_corr.loc[ ['LARGEST_ORDER_SIZE', 'log_LARGEST_ORDER_SIZE'],
              ['REVENUE', 'log_REVENUE']   ]

In [ ]:
# AVG_CLICKS_PER_VISIT 

df['log_AVG_CLICKS_PER_VISIT'] = np.log10(df['AVG_CLICKS_PER_VISIT'])

log_corr = df.loc[ : , ['AVG_CLICKS_PER_VISIT',
                        'log_AVG_CLICKS_PER_VISIT',
                        'REVENUE',
                        'log_REVENUE']  ].corr(method = 'pearson')\
                                                 .round(decimals = 2)

log_corr.loc[ ['AVG_CLICKS_PER_VISIT', 'log_AVG_CLICKS_PER_VISIT'],
              ['REVENUE', 'log_REVENUE']   ]

In [ ]:
# PC_LOGINS        

df['log_PC_LOGINS'] = np.log10(df['PC_LOGINS'])

log_corr = df.loc[ : , ['PC_LOGINS',
                        'log_PC_LOGINS',
                        'REVENUE',
                        'log_REVENUE']  ].corr(method = 'pearson')\
                                                 .round(decimals = 2)

log_corr.loc[ ['PC_LOGINS', 'log_PC_LOGINS'],
              ['REVENUE', 'log_REVENUE']   ]

In [ ]:
# PRODUCT_CATEGORIES_VIEWED

df['log_PRODUCT_CATEGORIES_VIEWED'] = np.log10(df['PRODUCT_CATEGORIES_VIEWED'])

log_corr = df.loc[ : , ['PRODUCT_CATEGORIES_VIEWED',
                        'log_PRODUCT_CATEGORIES_VIEWED',
                        'REVENUE',
                        'log_REVENUE']  ].corr(method = 'pearson')\
                                                 .round(decimals = 2)

log_corr.loc[ ['PRODUCT_CATEGORIES_VIEWED', 'log_PRODUCT_CATEGORIES_VIEWED'],
              ['REVENUE', 'log_REVENUE']   ]

## Combine Interval/Count Features 

In [ ]:
# TOTAL DELIVERIES 

# Step 1: combine features 
df['TOTAL_DELIVERY'] = df['EARLY_DELIVERIES'] + df['LATE_DELIVERIES']


# Step 2: develop a correlation matrix 
delivery_corr = df.loc[ : , ['EARLY_DELIVERIES',
                              'LATE_DELIVERIES',
                              'TOTAL_DELIVERY',
                              'REVENUE']  ].corr(method = 'pearson')\
                                              .round(decimals = 2)

# Step 3: print value counts and correlations deliveries
print(f"""
---------------------
Correlations
---------------------
{delivery_corr['REVENUE']}

""")

In [ ]:
# TOTAL LOCKERS 

# Step 1: combine features 
df['TOTAL_LOCKERS'] = df['REFRIGERATED_LOCKER'] + df['PACKAGE_LOCKER']

# Step 2: develop a correlation matrix 
locker_corr = df.loc[ : , ['REFRIGERATED_LOCKER',
                              'PACKAGE_LOCKER',
                              'TOTAL_LOCKERS',
                              'REVENUE']  ].corr(method = 'pearson')\
                                              .round(decimals = 2)

# Step 3: print value counts and correlations deliveries
print(f"""
---------------------
Correlations
---------------------
{locker_corr['REVENUE']}

""")

In [ ]:
# TOTAL CANCELLATION 

# Step 1: combine features 
df['TOTAL_CANCELLATION'] = df['CANCELLATIONS_BEFORE_NOON'] + \
                            df['CANCELLATIONS_AFTER_NOON']


# Step 2: develop a correlation matrix 
cancel_corr = df.loc[ : , ['CANCELLATIONS_BEFORE_NOON',
                              'CANCELLATIONS_AFTER_NOON',
                              'TOTAL_CANCELLATION',
                              'REVENUE']  ].corr(method = 'pearson')\
                                              .round(decimals = 2)

# Step 3: print value counts and correlations deliveries
print(f"""
---------------------
Correlations
---------------------
{cancel_corr['REVENUE']}

""")

## Engineer Categorical Data

### Assign Labels and One Hot Encode

In [ ]:
# MASTER_CLASSES_ATTENDED

# Step 1: assign labels to data 
df['MASTER_CLASS_ATTENDED_TIERS'] = pd.cut(df['MASTER_CLASSES_ATTENDED'], bins=[-1,0,1,2,3],\
                           labels = ["Never","Rarely", "Occasionally", "Regularly"])

# Step 2: one hot encode variable
one_hot_attendance = pd.get_dummies(df['MASTER_CLASS_ATTENDED_TIERS'])

# Step 3: drop variable after they've been encoded
df = df.drop('MASTER_CLASS_ATTENDED_TIERS', axis = 1)

# Step 4: join coding together
df = df.join([one_hot_attendance])

# Step 5: save new columns
new_columns = df.columns

In [ ]:
# MEDIAN_MEAL_RATING

# Step 1: assign labels to data 
df['MEDIAN_MEAL_RATING_TIERS'] = pd.cut(df['MEDIAN_MEAL_RATING'], bins=[-1,0,1,2,3,4,5],\
                        labels = ["Terrible","Bad","Decent", "Good", "Great", "Excellent"])

# Step 2: one hot encode variable
one_hot_tiers    = pd.get_dummies(df['MEDIAN_MEAL_RATING_TIERS'])

# Step 3: drop variable after they've been encoded
df = df.drop('MEDIAN_MEAL_RATING_TIERS', axis = 1)

# Step 4: join coding together
df = df.join([one_hot_tiers])

# Step 5: save new columns
new_columns = df.columns

### Split EMAILS into Domains and One Hot Encode

In [ ]:
# Step 1: split EMAILS

# create an empty placeholder list
placeholder_lst = []

# loop over each email address
for index, col in df.iterrows():
    
    # split email domain at '@'
    split_email = df.loc[index, 'EMAIL'].split(sep = '@')
    
    # append placeholder_lst with the results
    placeholder_lst.append(split_email)
    
# convert placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)

In [ ]:
# Step 2: concatenate with original DataFrame

# rename column to concatenate
email_df.columns = ['account_name' , 'domain'] 

# concatenate 'domain' to  original DataFrame 
df = pd.concat([df, email_df['domain']],   
                   axis = 1)

In [ ]:
# Step 3: aggregate emails into groups

# specify domain types 
professional = [    '@mmm.com',
                    '@amex.com',
                    '@apple.com',
                    '@boeing.com',
                    '@caterpillar.com',
                    '@chevron.com', 
                    '@cisco.com', 
                    '@cocacola.com', 
                    '@disney.com', 
                    '@dupont.com', 
                    '@exxon.com', 
                    '@ge.org',
                    '@goldmansacs.com',
                    '@homedepot.com', 
                    '@ibm.com', 
                    '@intel.com', 
                    '@jnj.com',
                    '@jpmorgan.com',
                    '@mcdonalds.com', 
                    '@merck.com', 
                    '@microsoft.com',
                    '@nike.com', 
                    '@pfizer.com', 
                    '@pg.com', 
                    '@travelers.com',
                    '@unitedtech.com',
                    '@unitedhealth.com', 
                    '@verizon.com', 
                    '@visa.com', 
                    '@walmart.com'       ]

personal   = [      '@gmail.com', 
                    '@yahoo.com', 
                    '@protonmail.com'    ]

junk       = [      '@me.com',
                    '@aol.com',
                    '@hotmail.com', 
                    '@live.com', 
                    '@msn.com',
                    '@passport.com'      ]

# create an empty placeholder list
placeholder_lst = []


# loop to group observations by domain type
for domain in df['domain']:
        if '@' + domain in personal:
            placeholder_lst.append('professional')
                  
        elif '@' + domain in junk:
            placeholder_lst.append('personal')
        
        elif '@' + domain in professional:
            placeholder_lst.append('junk')
            
        else:
            print('Unknown')

# concatenate with original DataFrame
df['group_domain'] = pd.Series(placeholder_lst)

In [ ]:
# Step 4: get dummy variables for 'domain'

# one hot encode variable 
one_hot_domain    = pd.get_dummies(df['group_domain'])

# drop variables after they've been encoded
df = df.drop('group_domain', axis = 1)

# join coding together
df = df.join([one_hot_domain])

# save new columns
new_columns = df.columns

# TEST MODEL PERFORMANCE

## ORDINARY LEAST SQUARES (OLS) MODEL

### Test using full dataset with original Y-variable

In [ ]:
# Step 1: Prepare data for train/split test 
# prepare explanatory variables by dropping objects and y-variables
df_data   = df.drop(['REVENUE',
                    'log_REVENUE',
                    'NAME', 
                    'EMAIL', 
                    'FIRST_NAME', 
                    'FAMILY_NAME',
                    'domain'], axis = 1)

# prepare response variables
df_target =df.loc[ : , 'REVENUE']
log_df_target =df.loc[ : , 'log_REVENUE']

# Step 2: set up train-test split 
x_train_FULL, x_test_FULL, y_train_FULL, y_test_FULL = train_test_split(
            df_data,     # x-variables
            df_target,   # original y-variable
            test_size = 0.25,
            random_state = 219)

# Step 3: double check the shapes of the dataset
print(f"""
Training Data
-------------
X-side: {x_train_FULL.shape}
y-side: {y_train_FULL.shape}


Testing Data
------------
X-side: {x_test_FULL.shape}
y-side: {y_test_FULL.shape}
""")

In [ ]:
# Step 4: conduct test

# instantiate a model object
lr = LinearRegression()

# fit to the training data
lr_fit = lr.fit(x_train_FULL, y_train_FULL)

# predict on new data
lr_pred = lr_fit.predict(x_test_FULL)

# score the results
print('OLS Training Score :', lr.score(x_train_FULL, y_train_FULL).round(4))  
print('OLS Testing Score  :',  lr.score(x_test_FULL, y_test_FULL).round(4)) 

# save scores for future use 
lr_train_score = lr.score(x_train_FULL, y_train_FULL).round(4) 
lr_test_score  = lr.score(x_test_FULL, y_test_FULL).round(4)   

# display and save the gap between training and testing
print('OLS Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

### Test using full dataset with transformed Y-variable

In [ ]:
# Step 1: Prepare data for train/split test 
# prepare explanatory variables by dropping objects and y-variables
df_data   = df.drop(['REVENUE',
                    'log_REVENUE',
                    'NAME', 
                    'EMAIL', 
                    'FIRST_NAME', 
                    'FAMILY_NAME',
                    'domain'], axis = 1)

# prepare response variables
df_target =df.loc[ : , 'REVENUE']
log_df_target =df.loc[ : , 'log_REVENUE']

# Step 2: set up train-test split 
x_train_log, x_test_log, y_train_log, y_test_log = train_test_split(
            df_data,         # x-variables
            log_df_target,   # transformed y-variable
            test_size = 0.25,
            random_state = 219)

# Step 3: double check the shapes of the dataset
print(f"""
Training Data
-------------
X-side: {x_train_log.shape}
y-side: {y_train_log.shape}


Testing Data
------------
X-side: {x_test_log.shape}
y-side: {y_test_log.shape}
""")

In [ ]:
# Step 4: conduct test

# instantiate a model object
lr = LinearRegression()

# fit to the training data
lr_fit = lr.fit(x_train_log, y_train_log)

# predict on new data
lr_pred = lr_fit.predict(x_test_log)

# score the results
print('OLS Training Score :', lr.score(x_train_log, y_train_log).round(4)) 
print('OLS Testing Score  :',  lr.score(x_test_log, y_test_log).round(4))

# save scores for future use 
lr_train_score = lr.score(x_train_log, y_train_log).round(4)
lr_test_score  = lr.score(x_test_log, y_test_log).round(4)  

# display and save the gap between training and testing
print('OLS Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

#### Find model efficients and variables

In [ ]:
# zip each feature name to its coefficient
lr_model_values = zip(df_data.columns,
                      lr_fit.coef_.round(decimals = 2))

# set up a placeholder list to store model features
lr_model_lst = [('intercept', lr_fit.intercept_.round(decimals = 2))]

# print out each feature-coefficient pair one by one
for val in lr_model_values:
    lr_model_lst.append(val)
    
# check the results
for pair in lr_model_lst:
    print(pair)

# FINAL MODEL OUTPUT

In [ ]:
# create a dictionary of OLS model results
model_performance = {
    
    'Model Type'    : ['OLS'],
           
    'Training' : [lr_train_score],
           
    'Testing'  : [lr_test_score],
                    
    'Train-Test Gap' : [lr_test_gap],
                    
    'Model Size' : [len(lr_model_lst)],
                    
    'Model' : [lr_model_lst]}

# convert model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)

# send model results to Excel
model_performance.to_excel('./model_results/assignment_linear_model_performance.xlsx',
                           index = False)

# read final model output
pd.read_excel('./model_results/assignment_linear_model_performance.xlsx')